# Parsing OLAP reports

[OLAP](https://en.wikipedia.org/wiki/Online_analytical_processing) report format is intented to be a result of code, not an input. However, sometimes there is no other option except parsing such files. And here is an example!

OLAP reports have several features that make such tables hard to parse with existing tools:  
0. Many columns are grouped ones. It means that their values are sparse, and must be used for rows below.
0. Tables have aggregated rows which must be removed.
0. Upper-header rows which should be used to distinguish columns with the same name.
0. Padding rows on the top.

And all these features are handled by given parser! It can easily deal with given OLAP report with CSV, TSV or XLS extension. Moreover, it has different settings and is very customisable allowing you to control all the details – because different companies and apps create different OLAP reports with different column names, padding rows number and other features.

0k, let's have a look on our exemplary report [`olap.csv`](https://github.com/AivanF/Lemuras/blob/master/examples/data/olap.csv):

![Area](data/olap-screen.png)

To create custom parser class, you need to inherit **`OlapParser`** which provides you with all the flexible stuff. It's instances have `.parse(fl, filename)` method for converting an OLAP report to a usual Lemuras table.

In [1]:
from olap_parser import OlapParser

To describe columns, you should use **`ColumnData`** class (which is a named tuple under the hood). And the simplest way to create it is to use `ColumnData_make(**kwargs)` function. It has the following arguments:

- **`name`** – just a string name of final column. Required field.
- **`type`** – type which will be used to convert cell values. It must be a string with type name (one of Lemuras supported types, they described in `Example 1.5 - Functions Applying`), or your own executable that takes a string and returns anything. Required field.
- **`cache`** – if `True`, it indicates that the columns is a grouped one. It's values will be saved and used instead of empty cells. Otherwise, empty cells will be filled with the default value. `cache=True` by default.
- **`default`** – default value for empty cells of grouped column. `default=None` by default.

In [2]:
from olap_parser import ColumnData_make

If parser meets predictable troubles, it throws **`ParserError`** exceptions which means that given file is not supported. By the way, you can freely use it in custom parsers.

In [3]:
from olap_parser import ParserError

## Step 1. Parsing rules definition

In [4]:
class CustomParser(OlapParser):
    def __init__(self):
        super().__init__()
        # The first row contains useless info
        self.skip_header_rows = 1
        # It will help distinguish Discount Sum and Total Sum
        self.meta_header_rows = 1

    def check_skip(self, line, row):
        # line arg is a string, and row arg is a list of cells

        # Don't skip if current row is a header
        if self.columns is None:
            return False

        # Skip aggregated rows
        line = line.lower()
        return len(line) == 0 or 'total' in line

    def describe_column(self, column_index, name, meta):
        # Should return None or ColumnData object
        name = name.lower()

        if name == 'country':
            return ColumnData_make(name='country', type='str', cache=True)

        if name == 'reported':
            return ColumnData_make(name='created', type='date', cache=True)
        
        # Note that meta headers spread to their right
        # So that Discount meta header is the same for 2 columns
        if 'sum' in name and 'Discount' in meta:
            return ColumnData_make(name='discount_sum', type='int', cache=False)
        
        if 'sum' in name and 'Total' in meta:
            return ColumnData_make(name='total_sum', type='int', cache=False)
        
        if 'order count' in name and 'Total' in meta:
            return ColumnData_make(name='count', type='int', cache=False)

        def parse_category(raw_value):
            raw_value = raw_value.lower()
            if 'vegetables' in raw_value or 'fruits' in raw_value:
                return 1
            if 'meat' in raw_value or 'fish' in raw_value:
                return 2
            return 3

        if 'category' in name:
            return ColumnData_make(name='category', type=parse_category, cache=False)

        def parse_pay_type(raw_value):
            raw_value = raw_value.lower()
            if 'transfer' in raw_value:
                return 1
            if 'cash' in raw_value:
                return 2
            if 'debt' in raw_value:
                return 3
            return 0

        if 'payment' in name:
            return ColumnData_make(name='pay_type', type=parse_pay_type, cache=False)

        return None

    def validate_columns(self):
        # Basic check of necessary columns
        need_columns = ['country', 'category', 'created', 'total_sum']
        for column in need_columns:
            if column not in self.res_columns:
                raise ParserError('Column "{}"" not found!'.format(column))

In [5]:
# Some visual trick, never mind...
from lemuras import Table ; Table.maxshowrows = 16

That's it! We have just described what we see in the report and what we wanna get from it. Let's try it out.

## Step 2. Usage

In [6]:
parser = CustomParser()
df = parser.parse(None, 'data/olap.csv')
df

country,category,created,discount_sum,count,total_sum,pay_type
Austria,3,2019-04-25,93,28,3080,1
Germany,1,2019-04-27,33,18,1620,2
Germany,2,2019-04-27,0,20,2400,1
Germany,1,2019-04-28,46,23,2300,2
Germany,2,2019-04-28,68,14,1680,1
Poland,1,2019-04-29,60,12,1200,1
Poland,1,2019-04-29,36,16,1760,1
Poland,2,2019-04-29,72,20,2400,3
Ukraine,3,2019-04-18,130,26,2600,3
Spain,2,2019-04-24,0,33,3960,2


Absolutely correct Lemuras Table! Now you can parse OLAP reports really fast and easily `:)`

If you need more flexibility, have a look at [**`OlapParser.init`** method](https://github.com/AivanF/Lemuras/blob/master/examples/olap_parser.py#L36), there are several helpful variables.